In [8]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import pathlib

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display

from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

In [9]:
# patch tf1 into `utils.ops`
utils_ops.tf = tf.compat.v1

# Patch the location of gfile
tf.gfile = tf.io.gfile

In [10]:
import pathlib 

def load_custom_model(model_name):
  model_file = model_name

  model_dir = pathlib.Path(model_file)/"saved_model"

  model = tf.saved_model.load(str(model_dir))

  return model

In [11]:
# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = 'object_detection/training/labelmap.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

In [12]:
# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
PATH_TO_TEST_IMAGES_DIR = pathlib.Path('object_detection/testing')
TEST_IMAGE_PATHS = sorted(list(PATH_TO_TEST_IMAGES_DIR.glob("*.jpg")))
TEST_IMAGE_PATHS

[WindowsPath('object_detection/testing/11.jpg'),
 WindowsPath('object_detection/testing/425.jpg'),
 WindowsPath('object_detection/testing/65.jpg'),
 WindowsPath('object_detection/testing/72.jpg'),
 WindowsPath('object_detection/testing/83.jpg'),
 WindowsPath('object_detection/testing/99.jpg'),
 WindowsPath('object_detection/testing/shoo3.jpg'),
 WindowsPath('object_detection/testing/Warehouse-Gloves_1.jpg'),
 WindowsPath('object_detection/testing/warehouse-safety-float1.jpg'),
 WindowsPath('object_detection/testing/wh_dresscode_0004.jpg'),
 WindowsPath('object_detection/testing/wh_dresscode_0005.jpg'),
 WindowsPath('object_detection/testing/wh_dresscode_0030.jpg'),
 WindowsPath('object_detection/testing/wh_dresscode_0031.jpg'),
 WindowsPath('object_detection/testing/wh_dresscode_0035.jpg'),
 WindowsPath('object_detection/testing/wh_dresscode_0045.jpg'),
 WindowsPath('object_detection/testing/wh_dresscode_0054.jpg'),
 WindowsPath('object_detection/testing/wh_dresscode_0079.jpg'),
 Windo

In [ ]:
model_name = 'object_detection/exported-models/my_model'
detection_model = load_custom_model(model_name)

In [ ]:
def run_inference_for_single_image(model, image):
  image = np.asarray(image)
  # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
  input_tensor = tf.convert_to_tensor(image)
  # The model expects a batch of images, so add an axis with `tf.newaxis`.
  input_tensor = input_tensor[tf.newaxis,...]

  # Run inference
  output_dict = model(input_tensor)

  # All outputs are batches tensors.
  # Convert to numpy arrays, and take index [0] to remove the batch dimension.
  # We're only interested in the first num_detections.
  num_detections = int(output_dict.pop('num_detections'))
  output_dict = {key:value[0, :num_detections].numpy() 
                 for key,value in output_dict.items()}
  output_dict['num_detections'] = num_detections

  # detection_classes should be ints.
  output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)
   
  # Handle models with masks:
  if 'detection_masks' in output_dict:
    # Reframe the the bbox mask to the image size.
    detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
              output_dict['detection_masks'], output_dict['detection_boxes'],
               image.shape[0], image.shape[1])      
    detection_masks_reframed = tf.cast(detection_masks_reframed > 0.7,
                                       tf.uint8)
    output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()
    
  return output_dict

In [ ]:
def show_inference(model, image_path):
  # the array based representation of the image will be used later in order to prepare the
  # result image with boxes and labels on it.
  image_np = image_path
  image_np=cv2.cvtColor(image_np,cv2.COLOR_BGR2RGB)
  # Actual detection.
  output_dict = run_inference_for_single_image(model, image_np)
  # Visualization of the results of a detection.
  vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks_reframed', None),
      use_normalized_coordinates=True,
      line_thickness=2)
  print(output_dict)
  print(category_index)
  image_np=cv2.cvtColor(image_np,cv2.COLOR_BGR2RGB)
  color_detect(image_np,output_dict['detection_classes'],output_dict['detection_scores'],output_dict['detection_boxes'])
  return image_np

In [ ]:
# BLUE RED YELLOW GREEN BLACK


import numpy as np 
from scipy import stats
import pandas as pd


def color_detect(image,classes,score,boxes):
    
    for i in range(100):
        if(classes[i]==2 and score[i]>0.7):       #colour detection will done when the prediction is above 70%
            
            h,w=image.shape[0:2]
            #image.shape=[height,width,3]
            
#coordinates of bounding boxes
            
            ymin,xmin,ymax,xmax=boxes[i]
            
            
            x1 = int(xmin * w)
            x2 = int(xmax * w)
            y1 = int(ymin * h)
            y2 = int(ymax * h)
    
            x = x2-x1
            y = y2-y1
            
            
            start_point = int(xmin * w), int(ymin * h)
            end_point = int(xmax * w), int(ymax * h)
            color = (0, 0, 0)
            thickness = 2
            radius =  int(min(x,y)/2)

            
            center=(int(((xmin+xmax)/2)*w),int(((ymin+ymax)/2)*h))
            #circle = cv2.circle(image,center,radius,(0,0,255),3)
            
#tracking bounding boxes
             
            rect = cv2.rectangle(image, start_point,  end_point, color, thickness)
          
            circle_img = np.zeros((h, w), np.uint8)

         
            mask = cv2.circle(circle_img, center, radius, 1, thickness=-1)
            masked_img = cv2.bitwise_and(rect, rect, mask=circle_img)

            circle_locations = mask == 1
            bgr = img[circle_locations]

            rgb = bgr[..., ::-1]
            
            max_rgb =  np.average(rgb, axis=0)
     
            #print("XY-Cordinate : ",x,y)
            
            xpos = x
            ypos = y
            r, g, b = max_rgb
            b = int(b)
            g = int(g)
            r = int(r)
            
            print( "r: ",r,"g: ",g,"b: ",b)
            
            
           
            
            r, g, b = r/255.0, g/255.0, b/255.0
            mx = max(r, g, b)
            mn = min(r, g, b)
            diff = mx-mn
            if mx == mn:
                h_hsv = 0 
            elif mx == r:
                h_hsv = (60 * ((g-b)/diff) + 360) % 360   
            elif mx == g:
                h_hsv = (60 * ((b-r)/diff) + 120) % 360   
            elif mx == b:
                h_hsv = (60 * ((r-g)/diff) + 240) % 360   
            if mx == 0:
                s_hsv = 0
            else:
                s_hsv = (diff/mx)*100   
            v_hsv = mx*100

            print("HSV VALUE", int(h_hsv))
            
            
            font = cv2.FONT_HERSHEY_SIMPLEX 
            org = (int(xmax * w),int(ymax * h))
            fontScale = 0.5
            color = (0,0,0)
            thickness = 3
            
#for vest colour
            

            if (0 <= h_hsv <=8):   #red
                print("red")
                cv2.putText(rect, "Manager", org, font,  fontScale, color, thickness, cv2.LINE_AA)
                cv2.putText(rect, "Manager" , org, font,  fontScale, (255,255,255), 1, cv2.LINE_AA)
                
            elif (9 <= h_hsv <=36):   #orange
                print("orange")
                cv2.putText(rect, "SR. Manager", org, font,  fontScale, color, thickness, cv2.LINE_AA)
                cv2.putText(rect, "SR. Manager" , org, font,  fontScale, (255,255,255), 1, cv2.LINE_AA)
                
            elif (37 <=  h_hsv <=79):   #yellow
                print("yellow")
                cv2.putText(rect, "Worker", org, font,  fontScale, color, thickness, cv2.LINE_AA)
                cv2.putText(rect, "Worker" , org, font,  fontScale, (255,255,255), 1, cv2.LINE_AA)
                
            elif (80 <= h_hsv <=149):    #green
                print("green")
                cv2.putText(rect, "Developer", org, font,  fontScale, color, thickness, cv2.LINE_AA)
                cv2.putText(rect, "Developer" , org, font,  fontScale, (255,255,255), 1, cv2.LINE_AA)
                
            elif (206 <= h_hsv <=258):     #blue
                print("blue")
                cv2.putText(rect, "Labour", org, font,  fontScale, color, thickness, cv2.LINE_AA)
                cv2.putText(rect, "Labour" , org, font,  fontScale, (255,255,255), 1, cv2.LINE_AA)
                
            else:
                cv2.putText(rect, "NONE", org, font,  fontScale, color, thickness, cv2.LINE_AA)
                cv2.putText(rect, "NONE" , org, font,  fontScale, (255,255,255), 1, cv2.LINE_AA)
            
            
           



    
      

In [7]:
import cv2

img=cv2.imread(r'object_detection/testing/wh_dresscode_0275.jpg')
img = cv2.resize(img,(1000,640))

img=show_inference(detection_model,img)
cv2.imshow('IMG',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

NameError: name 'show_inference' is not defined

In [ ]:
# Python program to save a 
# video using OpenCV 


import cv2 


# Create an object to read 
# from camera 
video = cv2.VideoCapture(r'F:\ware\vid1.mp4') 

# We need to check if camera 
# is opened previously or not 
if (video.isOpened() == False): 
	print("Error reading video file") 

# We need to set resolutions. 
# so, convert them from float to integer. 
frame_width = int(video.get(3)) 
frame_height = int(video.get(4)) 

size = (frame_width, frame_height) 

# Below VideoWriter object will create 
# a frame of above defined The output 
# is stored in 'filename.avi' file. 
result = cv2.VideoWriter('testing_video2.mp4', 
                        cv2.VideoWriter_fourcc(*'MJPG'), 
                        10, size) 

while(True): 
    ret,img=video.read()
    img=show_inference(detection_model,img)
    
    if ret == True: 

        # Write the frame into the 
        # file 'filename.avi' 
        result.write(img) 

        # Display the frame 
        # saved in the file 
        cv2.imshow('live',img) 

        # Press S on keyboard 
        # to stop the process 
        k=cv2.waitKey(1)
        if(k==27):
            break

    # Break the loop 
    else:
        break

# When everything done, release 
# the video capture and video 
# write objects 
video.release() 
result.release() 

# Closes all the frames 
cv2.destroyAllWindows() 

print("The video was successfully saved") 
